#Installing dependencies

In [1]:
!pip install -q langchain
!pip install -q torch
!pip install -q transformers
!pip install -q sentence-transformers
!pip install -q datasets
!pip install -q faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 927.4 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.6 MB/s eta 0:00:00


#Importing the essential libraries

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from transformers import AutoTokenizer, pipeline
from langchain import HuggingFacePipeline
from langchain.chains import RetrievalQA
from datasets import load_dataset

#Loading the remote data with the HuggingFace data loader

In [3]:
data_url = "https://huggingface.co/datasets/wikimedia/wikipedia/resolve/refs%2Fconvert%2Fparquet/20231101.en/train/0000.parquet?download=true"
data_files = {"train": data_url + "0000.parquet?download=true"}
text_wiki = load_dataset("parquet", data_files=data_files, split="train")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

#Inspecting the loaded data

In [4]:
text_wiki.to_pandas()

,id,url,title,text
0,12,https://en.wikipedia.org/wiki/Anarchism,Anarchism,Anarchism is a political philosophy and moveme...
1,39,https://en.wikipedia.org/wiki/Albedo,Albedo,Albedo (; ) is the fraction of sunlight that i...
2,290,https://en.wikipedia.org/wiki/A,A,"A, or a, is the first letter and the first vow..."
3,303,https://en.wikipedia.org/wiki/Alabama,Alabama,Alabama () is a state in the Southeastern regi...
4,305,https://en.wikipedia.org/wiki/Achilles,Achilles,"In Greek mythology, Achilles ( ) or Achilleus ..."
...,...,...,...,...
156284,5275580,https://en.wikipedia.org/wiki/Brookins%20Campbell,Brookins Campbell,"Brookins Campbell (1808December 25, 1853) was ..."
156285,5275584,https://en.wikipedia.org/wiki/Antonio%20Valencia,Antonio Valencia,"Luis Antonio Valencia Mosquera, known as Anton..."
156286,5275590,https://en.wikipedia.org/wiki/Geraint%20Bowen,Geraint Bowen,Geraint Bowen may refer to:\n\nGeraint Bowen (...
156287,5275592,https://en.wikipedia.org/wiki/Trstenik%20Airport,Trstenik Airport,"Trstenik Airport (, Latin: Aerodrom Trstenik)..."


#Checking for null values

In [5]:
text_wiki.to_pandas().isnull().sum()

id       0
url      0
title    0
text     0
dtype: int64

#Imlementing the RAG-based QA Pipeline with the LangChain framework and HuggingFace

Creating the embedding model

In [6]:
# Pretrained general use-case (light) HuggingFace model, for efficiency and also better choice for LangChain
model_route = "sentence-transformers/all-MiniLM-l6-v2"
model_kwargs = {'device':'cuda'} # Dict for model arguments
encode_kwargs = {'normalize_embeddings': False} # Setting it to false because there is no need to normalize them for cosine-similiraties.
# Howewer, if we were to use an euclidean-based algorithm, for eg.: K-means, normalizing would be neccessary

# Creating the HuggingFaceEmbedding from LangChain, with the above declared parameters
embeddings = HuggingFaceEmbeddings(
    model_name=model_route,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

Testing out the embedding model

In [7]:
text = "Test"
embedding = embeddings.embed_query(text)
print(embedding)

[0.011573387309908867, 0.025136232376098633, -0.03670187294483185, 0.059324879199266434, -0.007149024400860071, -0.04119419679045677, 0.0770874172449112, 0.03744250535964966, 0.012449018657207489, -0.006117619574069977, 0.0170342605561018, -0.07701531797647476, -0.00039419965469278395, 0.027909042313694954, -0.015989156439900398, -0.06827522069215775, 0.008884698152542114, -0.02028072439134121, -0.08035989850759506, -0.013074061833322048, -0.041099995374679565, -0.025898076593875885, -0.026538603007793427, 0.03305228799581528, -0.02207913063466549, 0.021046150475740433, -0.05792199820280075, 0.03294874727725983, 0.029707379639148712, -0.062248408794403076, 0.03878800570964813, 0.031990692019462585, 0.01533075887709856, 0.04530699551105499, 0.053149428218603134, 0.01336068008095026, 0.041224926710128784, 0.028142862021923065, 0.019398430362343788, -0.0032522918190807104, -0.0036123364698141813, -0.14286021888256073, 0.038071196526288986, -0.01091619674116373, 0.026093969121575356, 0.041

#Splitting the textual data into constant chunks to be more effective when it comes to feeding them into the model

In [8]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
docs = text_splitter.create_documents(text_wiki['text'][:15000])

# In this way, every text chunk will be embedded individually, and it could be a possible context for the question

#Creating a vector database for storing and quick retrieving the embedded documents

In [9]:
# Using Facebook's vector db
db = FAISS.from_documents(docs, embeddings)

In [10]:
# testing out the similiraty search in the db

question = "What is the alphabet"
lookup_res = db.similarity_search(question)
print(lookup_res[0].page_content)

An alphabet is a standardized set of basic written graphemes (called letters) representing phonemes, units of sounds that distinguish words, of certain spoken languages. Not all writing systems represent language in this way; in a syllabary, each character represents a syllable, and logographic systems use characters to represent words, morphemes, or other semantic units.


#Loading the LLM and it's corresponding tokenizer

In [11]:
model_name = "Intel/dynamic_tinybert"
tokenizer = AutoTokenizer.from_pretrained(model_name, padding=True, truncation=True, max_length=512)
# Tokenizing and transforming the input question to make it able to feed it into the model

# Creating a HuggingFace based pipeline
question_answerer = pipeline(
    "question-answering",
    model=model_name,
    tokenizer=tokenizer,
    return_tensors='pt'
)

# Creating a LangChain pipeline around the HuggingFace pipeline, and adding another parameters to it
llm = HuggingFacePipeline(
    pipeline=question_answerer,
    model_kwargs={"temperature": 0.7, "max_length": 512},
)

tokenizer_config.json:   0%|          | 0.00/351 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [12]:

# Making the vector db as a retriever object, making it possible to pass more paramters to the retriever, like getting back to top k most relevant documents as a context
retriever = db.as_retriever(search_kwargs={"k": 4})

# Wrapping the finel QA instance around the pipeline, with adding the llm, and the retriever
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="refine", retriever=retriever, return_source_documents=False)

#Running the RAG model

In [13]:
question = "Where is Alabama?"
result = qa.run({"query": question})
print(result["result"])

# Will give the result with a ValueError, it is a known issue for the qa pipeline according to the forums.

ValueError: ignored

#Javaslataim az eredmény prezentálására a végfelhasználó felé

In [ ]:
'''
Ezt a notebookot mindenképp valamilyen backend szerveren hagynám, és valamilyen frontendről jönne a request/szöveg, ha valaki kérdést tesz fel. Papermill segítségével lehet notebookot paraméterezni, (akárcsak Databricksben), továbbá
egy másik megoldás lehet pl. a kérdés fájlba vagy valahova kiiratása, amit aztán a notebook felolvas. Persze notebookot is lehet futtatni egyben, pipelineként, de igény szerint szét is lehetne szedni OOP struktúrába is. Mivel az ügyfél csak
a kapott válaszában érdekelt, ezért mindenképp csak a szöveget küldeném neki vissza arra a felületre a backendről, ahonnan a kérdést indította.

Továbbá egy-két gondolat az általános kiértékelésről. Alapvetően szerintem üzleti igénytől és területtől függ az eredmény prezentálásának a módja. Én ezt eltudom képzelni akár egy weboldalon, automatikus chatbotként, Angular/vagy sima vanilla javascript frontenddel, és mondjuk egy Flask backenddel.
Vagy egy szimpla facebook botként, ami a leghasznosabb lenne minden korosztály számára. A weboldalas megoldáshoz viszont nyilván kicsivel több erőforrás is szükséges a frontend és a backend miatt. Illetve a chatbotoknál szintén szükség van arra, hogy az üzenetek időrendi összefüggéseit is eltároljuk, amire viszont nem minden modell képes.

Továbbá egy fokkal formálisabb és egyszerűbb megoldás lehetne egy automatikus e-mail válasz a kérdésre, és minden e-mailt egymástól függetlenként tekinteni.
Illetve aminek szerintem még lehet relevanciája, az mégpedig egy automatikus sms-küldés. Ennek nyilván többlet költség vonzata van, viszont mivel mindenki használja a telefont, ez az opció internet nélkül is könnyedén megvalósítható lehetne a user szemszögéből.



'''

#A válaszok kiértékelésére tett javaslatom

In [ ]:

'''
Ha vannak labelek/előre megírt kérdés-válasz párok a tanítóadatban, akkor Exact Match vagy F1 score szerintem egy jó metrika lehet. Viszont ez a jelen esetben nem kivitelezhető, ezért egy jó alternatív megoldás lehet,
ha megnézzük a model probabilitiesből, hogy melyik válaszba mennyire biztos a modell, és alapvetően mennyire volt hasznos a válasz a szövegkörnyezetből, a szemantikából, illetve a logikai kapcsolatokból.

'''
